# Pandas - I/O y Análisis

Este notebook cubre la lectura y escritura de archivos con Pandas, análisis descriptivo de datos y manejo de valores faltantes (NaN).

## Lectura de Archivos


In [1]:
import pandas as pd

# Métodos de lectura: todos se llaman pd.read_* donde * es el tipo de archivo
# Uso: pd.read_clipboard() para extracciones únicas
#      Otros métodos en scripts para análisis repetibles

# CSV
# df = pd.read_csv('archivo.csv')
# df = pd.read_csv('archivo.csv', sep=',', encoding='utf-8')
# df = pd.read_csv('archivo.csv', header=1, skiprows=1, skipfooter=2)
# df = pd.read_csv('archivo.csv', index_col=0)  # Usar primera columna como índice
# df = pd.read_csv('archivo.csv', nrows=100)  # Solo primeras 100 filas
# df = pd.read_csv('archivo.csv', names=['Col1', 'Col2'])  # Establecer nombres de columnas
# df = pd.read_csv('archivo.csv', parse_dates=True)  # Parsear fechas
# df = pd.read_csv('archivo.csv', parse_dates=[0, 2])  # Parsear columnas específicas
# df = pd.read_csv('archivo.csv', parse_dates={'Date': [0, 2]})  # Agrupar columnas como fecha
# df = pd.read_csv('archivo.csv', na_values=['-', 'N/A'])  # Valores a tratar como NaN
# df = pd.read_csv('archivo.csv', comment='#')  # Carácter que indica línea comentada
# df = pd.read_csv('archivo.csv', chunksize=1000)  # Leer en chunks

# read_table (más general que read_csv)
# pd.read_table('archivo.txt', sep='\t')  # Similar a read_csv pero por defecto sep='\t'

# Excel
# df = pd.read_excel('archivo.xlsx', sheet_name='Hoja1')
# df = pd.read_excel('archivo.xlsx', sheet_name=0)  # Primera hoja

# JSON
# df = pd.read_json('archivo.json')
# df = pd.read_json('archivo.json', orient='records')

# Parquet
# df = pd.read_parquet('archivo.parquet')

# HTML (tablas) - retorna lista de DataFrames
# dfs = pd.read_html('url_o_archivo.html')
# df = dfs[0]  # Primer DataFrame

# Ejemplo con datos de ejemplo (StringIO)
from io import StringIO
datos_csv = "nombre,edad\nAna,25\nJuan,30"
df = pd.read_csv(StringIO(datos_csv))
print(df)


  nombre  edad
0    Ana    25
1   Juan    30


## Escritura de Archivos


In [2]:
# Métodos de escritura: todos se llaman s_df.to_* donde * es el tipo de archivo

# CSV
# df.to_csv('archivo.csv', index=False)
# df.to_csv('archivo.csv', sep=';', encoding='utf-8')
# s.to_csv('archivo.csv')  # Series también tienen to_csv

# Excel (requiere openpyxl: pip install openpyxl)
# df.to_excel('archivo.xlsx', sheet_name='Hoja1', index=False)
# Escribir múltiples DataFrames a un solo archivo Excel:
# writer = pd.ExcelWriter('archivo.xlsx')
# df1.to_excel(writer, sheet_name='First')
# df2.to_excel(writer, sheet_name='Second')
# writer.save()

# JSON
# df.to_json('archivo.json')
# df.to_json('archivo.json', orient='records')

# Parquet (requiere pyarrow o fastparquet)
# df.to_parquet('archivo.parquet')

# HDF5 (requiere pytables)
# df.to_hdf('archivo.h5', 'key')
# df = pd.read_hdf('archivo.h5', 'key')

# Base de datos (requiere SQLAlchemy)
# from sqlalchemy import create_engine
# engine = create_engine('sqlite:///database.db')
# conn = engine.connect()
# df.to_sql('table_name', conn)  # Escribir
# df = pd.read_sql('SELECT * FROM table_name', conn)  # Leer
# df = pd.read_sql('table_name', conn)  # Leer tabla completa

# Ejemplo de escritura a StringIO (para pruebas)
from io import StringIO
output = StringIO()
df.to_csv(output, index=False)
print("CSV generado:")
print(output.getvalue())


CSV generado:
nombre,edad
Ana,25
Juan,30



## Análisis Descriptivo


In [3]:
df = pd.DataFrame({
    'A': [1, 2, 3, 4, 5],
    'B': [10, 20, 30, 40, 50]
})

# Estadísticas descriptivas (reducción - aplicadas columna por columna por defecto)
df.describe()          # Resumen estadístico (count, mean, std, min, 25%, 50%, 75%, max)
df.mean()              # Media por columna (axis=0 por defecto, across rows)
df.median()            # Mediana
# df.mad()               # Mean absolute deviation (deprecado/eliminado en pandas recientes)
df.std()               # Desviación estándar (Bessel-corrected sample std)
df.var()               # Varianza (unbiased variance)
df.sem()               # Standard error of the mean
df.skew()              # Sample skewness (3rd moment)
df.kurt()              # Sample kurtosis (4th moment)
df.quantile(0.5)       # Sample quantile (percentil 50 = mediana)
df.min()               # Mínimo
df.max()               # Máximo
df.sum()               # Suma
df.prod()              # Producto
df.count()             # Conteo de valores no nulos
df.value_counts()      # Conteo de valores únicos (solo Series)

# Operar por filas (axis=1 o axis='columns')
df.mean(axis=1)        # Media por fila
df.sum(axis='columns')  # Suma por fila

# Información del DataFrame
df.info()              # Tipos y memoria
df.head()              # Primeras 5 filas
df.tail()              # Últimas 5 filas
df.shape               # (filas, columnas)
df.columns.tolist()    # Lista de columnas
df.index               # Índice
df.values              # Array NumPy con datos
df.dtypes              # Tipos de datos por columna
len(df)                # Número de filas


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   A       5 non-null      int64
 1   B       5 non-null      int64
dtypes: int64(2)
memory usage: 212.0 bytes


5

## Manejo de Valores Faltantes


In [4]:
import numpy as np

# Valores faltantes: NaN (Not a Number, float64) o NaT (Not a Time)
# - Se propagan en operaciones (1 + NaN = NaN)
# - Se ignoran "sensatamente" en computaciones (sum ignora NaN, mean ignora NaN)
# - Permanecen NaN con operaciones matemáticas (np.log(NaN) = NaN)

# Crear DataFrame de ejemplo con valores faltantes
df_nan = pd.DataFrame({
    'A': [1, 2, np.nan, 4, 5],
    'B': [10, np.nan, 30, 40, 50]
})

# Detectar valores faltantes
df_nan.isna()              # DataFrame booleano
df_nan.isnull()            # Alias de isna()
pd.isnull(df_nan)          # Función equivalente
df_nan.notna()             # Valores no nulos
df_nan.notnull()           # Alias de notna()
pd.notnull(df_nan)         # Función equivalente
df_nan.isna().sum()        # Conteo de NaN por columna

# Reemplazar valores faltantes
# Usar fillna() o asignación directa con máscara booleana
df_nan[df_nan.isnull()] = 0  # Reemplazar NaN con 0 usando máscara
# O usar: df_nan.fillna(0, inplace=True)

# Eliminar valores faltantes
df_nan.dropna()            # Eliminar filas si cualquier valor es NaN (how='any')
df_nan.dropna(how='any')   # Eliminar filas si cualquier valor es NaN
df_nan.dropna(how='all')   # Eliminar filas si todos los valores son NaN
df_nan.dropna(axis=1)      # Eliminar columnas con NaN
df_nan.dropna(axis=1, how='all')  # Eliminar columnas si todos son NaN
df_nan.dropna(subset=['A'])  # Eliminar solo si 'A' es NaN

# Rellenar valores faltantes
df_nan2 = pd.DataFrame({
    'A': [1, 2, np.nan, 4, 5],
    'B': [10, np.nan, 30, 40, 50]
})
df_nan2.fillna(0)            # Rellenar con 0
df_nan2.ffill()              # Forward fill (último valor válido hacia adelante)
df_nan2.bfill()              # Backward fill (siguiente valor válido hacia atrás)
# df.fillna(method='ffill')  # Equivalente a ffill (deprecado, usar ffill())
# df.fillna(method='bfill')  # Equivalente a bfill (deprecado, usar bfill())
df_nan2['A'].fillna(df_nan2['A'].mean())  # Rellenar con media
df_nan2.interpolate(method='linear')  # Interpolar usando diferentes métodos


,A,B
0,1.0,10.0
1,2.0,20.0
2,3.0,30.0
3,4.0,40.0
4,5.0,50.0
